In [1]:
import numpy as np
import requests
import pandas as pd
import io
import warnings
warnings.filterwarnings("ignore")

### Para el Scraping Se utilizó Selenium junto con EdgeDriver para interactuar con la página del INDEC debido a:
- Contenido dinámico cargado por JavaScript. Los elementos de la página no están presentes en el HTML inicial.
- BeautifulSoup y requests no pueden ejecutar JavaScript ni interactuar con contenido dinámico.
- Selenium permite buscar elementos utilizando selectores CSS o XPath.
- EdgeDriver es necesario para que Selenium pueda controlar el navegador Edge y realizar acciones como abrir páginas.

En caso de utilizar Chrome, Moxila o otro navegador descargar y poner en tu path el driver.

1. Buscar el archivo, excepciones en caso de no encontrarlo

In [58]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Ruta personal a msedgedriver.exe, ajustar segun tu ruta al descargar.
ruta_msedgedriver = r"C:\WebDriver\msedgedriver.exe"
edge_service = Service(ruta_msedgedriver)

# Configurar Selenium para usar Edge, configurar en caso de utiliar otro navegador.
edge_options = Options()
edge_options.add_argument("--headless") 
driver = webdriver.Edge(service=edge_service, options=edge_options)

# URL base
url_indec = "https://www.indec.gob.ar/indec/web/Nivel4-Tema-3-5-31"

# Abrir la página
print("Abriendo la página del INDEC...")
driver.get(url_indec)

# Esperar a que la página cargue completamente
try:
    # Buscar el contenedor en la página
    print("Esperando a que el contenedor cargue...")
    contenedor = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.contSH.hide[id="1"]'))
    )
    print("Contenedor encontrado.")
    
    # Buscar el contenedor de texto que diga IPC y extraer su href
    texto_buscado = "Índice de precios al consumidor. Precios promedio de un conjunto de elementos de la canasta del IPC, según regiones"
    print(f"Texto encontrado: '{texto_buscado}'...")
    enlace_excel = contenedor.find_element(By.XPATH, f".//a[contains(text(), '{texto_buscado}')]")
    
    # Obtener la URL del archivo Excel
    url_selenium = enlace_excel.get_attribute("href")
    print(f"Enlace al archivo Excel encontrado: {url_selenium}")
    
except Exception as e:
    print(f"Error: {e}")

finally:
    # Cerrar el navegador al finalizar
    print("Cerrando el navegador...")
    driver.quit()

Abriendo la página del INDEC...
Esperando a que el contenedor cargue...
Contenedor encontrado.
Texto encontrado: 'Índice de precios al consumidor. Precios promedio de un conjunto de elementos de la canasta del IPC, según regiones'...
Enlace al archivo Excel encontrado: https://www.indec.gob.ar/ftp/cuadros/economia/sh_ipc_precios_promedio.xls
Cerrando el navegador...


extraer las hojas del archivo excel.

In [59]:
# En caso de no encontrar la url con selenium utilizar la url directa
url_scraping = url_selenium

# Obtener el archivo Excel
response = requests.get(url_scraping)

# Verificar la solicitud
if response.status_code == 200:
    # Leer el contenido del archivo Excel en un DataFrame
    indec_ipc = pd.read_excel(io.BytesIO(response.content), sheet_name=None, engine='xlrd')
    paginador = list(indec_ipc.keys())
    print(paginador)
else:
    print(f"Error al descargar el archivo: {response.status_code}")


['Nacional', 'GBA']


In [60]:
nacional = indec_ipc[paginador[0]] # Leer solamente la hoja Nacional.
comienzo = nacional.iloc[:5] # Ver el comienzo del dataset.
comienzo

,"Precios al consumidor de una selección de alimentos, bebidas y otros artículos de la canasta del Índice de precios al consumidor, según regiones. Junio 2017- noviembre 2024",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Región,Productos seleccionados,Unidad de medida,Año 2017,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre,...,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre
3,NaN,NaN,NaN,Pesos,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Identificar dinámicamente las filas de interés y ponerlas como nombre de columnas

In [61]:
# Buscar la fila que contiene los encabezados de las columnas
encabezados = ["Región", "Productos seleccionados", "Unidad de medida"]
fila_inicio = None

for i, fila in nacional.iterrows():
    if all(col in str(fila.values) for col in encabezados):
        fila_inicio = i
        break

if fila_inicio is not None:
    print(f"Encabezados encontrados en la fila: {fila_inicio}")
    # Establecer los encabezados como nombres de las columnas
    nacional.columns = nacional.iloc[fila_inicio]  # nombres de columnas
    # Leer los datos a partir de la fila de inicio
    nacional = nacional.iloc[fila_inicio + 1:] 
else:
    print("No se encontraron los encabezados esperados en el archivo.")


# Mostrar el DataFrame con los encabezados correctos
nacional.head()

Encabezados encontrados en la fila: 1


1,Región,Productos seleccionados,Unidad de medida,Año 2017,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre,...,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre
3,NaN,NaN,NaN,Pesos,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,GBA,Pan francés,kg,38.64,39.12,39.43,39.71,39.89,40.24,40.55,...,1942.55,2214.07,2376.84,2501.86,2597.5,2663.51,2774.95,2878.01,2948.01,3040.97
6,GBA,Harina de trigo común,kg,10.67,10.61,10.65,10.66,10.61,10.81,11.03,...,741.12,818.97,816.97,786.85,828.26,851.77,829.47,829.22,815.9,830.61


Hay 2 filas nulas antes de comenzar los datos. Que solo contienen el texto, "peso", es irrelevante para el analisis, seran eliminadas.

In [70]:
# Selecciona las filas 3 y 4
filas_seleccionadas = nacional.iloc[[1,2]]

# Identifica las columnas no nulas (en al menos una de las filas seleccionadas)
columnas_no_nulas = filas_seleccionadas.columns[filas_seleccionadas.notnull().any()]

# Filtra solo las columnas no nulas
resultado = filas_seleccionadas[columnas_no_nulas]

print("Columna donde hay al menos un valor:", columnas_no_nulas.tolist())
print("Valores:")
print(resultado)

# Elimina las filas 3 y 4 del DataFrame original
nacional = nacional.drop(index=[3, 4])
nacional.head(3)

Columna donde hay al menos un valor: ['Año 2017']
Valores:
1 Año 2017
3    Pesos
4      NaN


1,Región,Productos seleccionados,Unidad de medida,Año 2017,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre,...,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre
5,GBA,Pan francés,kg,38.64,39.12,39.43,39.71,39.89,40.24,40.55,...,1942.55,2214.07,2376.84,2501.86,2597.5,2663.51,2774.95,2878.01,2948.01,3040.97
6,GBA,Harina de trigo común,kg,10.67,10.61,10.65,10.66,10.61,10.81,11.03,...,741.12,818.97,816.97,786.85,828.26,851.77,829.47,829.22,815.9,830.61


Identificar la última fila válida

In [71]:
# Lista de regiones válidas
regiones_validas = ['GBA', 'Pampeana', 'Noreste', 'Noroeste', 'Cuyo', 'Patagonia']

# La última fila que contenga una región válida en la columna "Región"
ultima_fila_valida = nacional[nacional["Región"].isin(regiones_validas)].index[-1]

# Filtrar el DataFrame hasta la última fila válida
nacional = nacional.loc[:ultima_fila_valida]

nacional.tail(1)

1,Región,Productos seleccionados,Unidad de medida,Año 2017,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,Patagonia,Jabón de tocador,125 g,15.09,15.3,15.6,15.58,15.68,15.58,15.74,...,817.54,844.65,880.49,913.33,909.57,904.17,915.87,957.23,998.82,1011.4


Desplazar los datos de los meses 1 columna hacia la derecha a partir de la columna del año. Asi identificamos cada columna de año con su año para su pposterior transformación

In [72]:
# Insertar una columna nueva para no perder el ultimo dato 
nacional['temporal'] = 'temporal' 

# Insertar una columna para correr los datos y luego eliminarla
nacional.insert(3, 'temporal-d', np.nan)
nacional.iloc[:, 4:] = nacional.iloc[:, 3:-1].values
nacional = nacional.drop(nacional.columns[3], axis=1)  

# Asignar los años a la columna del año.
for año in range(2017, 2025):
    nacional[f'Año {año}'] = año
    
# Mostrar el DataFrame resultante
nacional.head(3)


1,Región,Productos seleccionados,Unidad de medida,Año 2017,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,temporal
2,NaN,NaN,NaN,2017,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,...,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre
5,GBA,Pan francés,kg,2017,38.64,39.12,39.43,39.71,39.89,40.24,...,1942.55,2214.07,2376.84,2501.86,2597.5,2663.51,2774.95,2878.01,2948.01,3040.97
6,GBA,Harina de trigo común,kg,2017,10.67,10.61,10.65,10.66,10.61,10.81,...,741.12,818.97,816.97,786.85,828.26,851.77,829.47,829.22,815.9,830.61


Verificacion de extraccion

In [74]:
# Extraer los años de las cabeceras de las columnas
years = nacional.columns[3:].values  # A partir de la cuarta columna
print(f'Años: {years}')
# Extraer los meses de la primera fila
months = nacional.iloc[0, 3:].values  # A partir de la cuarta columna
print(f'Meses: {months}')


Años: ['Año 2017' nan nan nan nan nan nan np.float64(nan) 'Año 2018' nan nan nan
 nan nan nan nan nan nan nan nan np.float64(nan) 'Año 2019' nan nan nan
 nan nan nan nan nan nan nan nan np.float64(nan) 'Año 2020' nan nan nan
 nan nan nan nan nan nan nan nan np.float64(nan) 'Año 2021' nan nan nan
 nan nan nan nan nan nan nan nan np.float64(nan) 'Año 2022' nan nan nan
 nan nan nan nan nan nan nan nan np.float64(nan) 'Año 2023' nan nan nan
 nan nan nan nan nan nan nan nan np.float64(nan) 'Año 2024' nan nan nan
 nan nan nan nan nan nan nan 'temporal']
Meses: [np.int64(2017) 'Junio' 'Julio' 'Agosto' 'Septiembre' 'Octubre'
 'Noviembre' 'Diciembre' np.int64(2018) 'Enero' 'Febrero' 'Marzo' 'Abril'
 'Mayo' 'Junio' 'Julio' 'Agosto' 'Septiembre' 'Octubre' 'Noviembre'
 'Diciembre' np.int64(2019) 'Enero' 'Febrero' 'Marzo' 'Abril' 'Mayo'
 'Junio' 'Julio' 'Agosto' 'Septiembre' 'Octubre' 'Noviembre' 'Diciembre'
 np.int64(2020) 'Enero ' 'Febrero' 'Marzo' 'Abril' 'Mayo' 'Junio' 'Julio'
 'Agosto' 'Septie

In [75]:
# Lista para almacenar las filas del DataFrame final
rows = []

# Mapeo manual de nombres de meses en español a números
meses_espanol = {
    "Enero": 1, "Febrero": 2, "Marzo": 3, "Abril": 4, "Mayo": 5, "Junio": 6,
    "Julio": 7, "Agosto": 8, "Septiembre": 9, "Octubre": 10, "Noviembre": 11, "Diciembre": 12
}

# Variable para almacenar el año actual
current_year = None

# Iterar sobre las filas de productos, regiones y unidades
for i in range(1, len(nacional)):  # Empezar desde la segunda fila (datos de precios)
    region = nacional.iloc[i, 0]  # Columna "Region"
    producto = nacional.iloc[i, 1]  # Columna "Productos seleccionados"
    unidad = nacional.iloc[i, 2]  # Columna "Unidad de medida"
    
    # Iterar sobre las columnas de años y meses
    for j in range(len(years)):
        year = years[j]  # Año correspondiente a la columna
        month = months[j]  # Mes correspondiente a la columna
        price = nacional.iloc[i, j + 3]  # Precio correspondiente (columna j + 3)
        
        # Si la columna es un año (por ejemplo, "Año 2017"), actualizar el año actual
        if isinstance(year, str) and year.startswith("Año "):
            try:
                current_year = int(year.replace("Año ", "").strip())
            except ValueError:
                print("Saltando columna: año no es un número válido.")  # Depuración
            continue  # Saltar esta columna, ya que no contiene un mes
        
        # Si el mes no es una cadena, saltar esta columna
        if not isinstance(month, str):
            print("Saltando columna: mes no es una cadena.")  # Depuración
            continue
        
        # Limpiar el nombre del mes (eliminar espacios adicionales)
        month_cleaned = month.strip().capitalize()
        
        # Convertir el mes a número usando el mapeo manual
        month_number = meses_espanol.get(month_cleaned)
        if month_number is None:
            print(f"Saltando columna: mes '{month_cleaned}' no reconocido.")  # Depuración
            continue
        
        # Si no hay un año actual, saltar esta columna
        if current_year is None:
            print("Saltando columna: no hay un año actual.")  # Depuración
            continue
        
        # Construir la fecha en formato yyyy-mm-dd (asumiendo el primer día del mes)
        date = f"{current_year}-{month_number:02d}-01"
        
        # Agregar una fila a la lista
        rows.append([region, producto, unidad, date, price])
        

# Crear el DataFrame final
final_df = pd.DataFrame(rows, columns=["Region", "Productos seleccionados", "Unidad de medida", "Date", "Price"])
# Reordenar y renombrar las columnas
final_df = final_df[["Date", "Region", "Productos seleccionados", "Unidad de medida", "Price"]]
final_df.columns = ["Date", "Region", "Product", "Unit", "Price"]

Agregar clave primaria a la tabla para manejarlo como identidad

In [77]:
final_df['product_id'] = range(1, len(final_df) + 1)
final_df

,Date,Region,Product,Unit,Price,product_id
0,2017-06-01,GBA,Pan francés,kg,38.64,1
1,2017-07-01,GBA,Pan francés,kg,39.12,2
2,2017-08-01,GBA,Pan francés,kg,39.43,3
3,2017-09-01,GBA,Pan francés,kg,39.71,4
4,2017-10-01,GBA,Pan francés,kg,39.89,5
...,...,...,...,...,...,...
7555,2024-07-01,Patagonia,Jabón de tocador,125 g,904.17,7556
7556,2024-08-01,Patagonia,Jabón de tocador,125 g,915.87,7557
7557,2024-09-01,Patagonia,Jabón de tocador,125 g,957.23,7558
7558,2024-10-01,Patagonia,Jabón de tocador,125 g,998.82,7559


### Validaciones y control de errores.

In [78]:
region_u = final_df['Region'].unique()
region_u
# Valores Correctos.

array(['GBA', 'Pampeana', 'Noreste', 'Noroeste', 'Cuyo', 'Patagonia'],
      dtype=object)

In [79]:
product_u = final_df['Product'].unique()
product_u
# Valores Correctos.

array(['Pan francés', 'Harina de trigo común', 'Arroz blanco simple',
       'Fideos secos tipo guisero', 'Carne picada común', 'Pollo entero',
       'Aceite de girasol', 'Leche fresca entera sachet',
       'Huevos de gallina', 'Papa', 'Azúcar', 'Detergente líquido',
       'Lavandina', 'Jabón de tocador'], dtype=object)

En caso de encontrar filas que no contenga la region o el producto la fila se elimina.

In [92]:
# Listas de regiones y productos válidos
regiones_validas = ['GBA', 'Pampeana', 'Noreste', 'Noroeste', 'Cuyo', 'Patagonia']
productos_validos = [
    'Pan francés', 'Harina de trigo común', 'Arroz blanco simple',
    'Fideos secos tipo guisero', 'Carne picada común', 'Pollo entero',
    'Aceite de girasol', 'Leche fresca entera sachet',
    'Huevos de gallina', 'Papa', 'Azúcar', 'Detergente líquido',
    'Lavandina', 'Jabón de tocador'
]

# Filtrar filas válidas - La fila debe tener una región válida Y un producto válido
filas_validas = final_df["Region"].isin(regiones_validas) & final_df["Product"].isin(productos_validos)
final_df = final_df[filas_validas]

# Redondear la columna "Price" a 2 decimales (si existe)
if "Price" in final_df.columns:
    final_df["Price"] = final_df["Price"].round(2)

# Mostrar el DataFrame filtrado
final_df

,Date,Region,Product,Unit,Price,product_id
0,2017-06-01,GBA,Pan francés,kg,38.64,1
2880,2017-06-01,Noreste,Carne picada común,kg,68.67,2881
2970,2017-06-01,Noreste,Pollo entero,kg,37.30,2971
3060,2017-06-01,Noreste,Aceite de girasol,"1,5 litros",51.08,3061
3150,2017-06-01,Noreste,Leche fresca entera sachet,Litro,22.53,3151
...,...,...,...,...,...,...
89,2024-11-01,GBA,Pan francés,kg,3040.97,90
1619,2024-11-01,Pampeana,Fideos secos tipo guisero,500 g,1381.12,1620
1799,2024-11-01,Pampeana,Pollo entero,kg,3228.91,1800
629,2024-11-01,GBA,Aceite de girasol,"1,5 litros",3295.82,630


In [93]:
unit_u = final_df['Unit'].unique()
unit_u
# Valores Correctos.

array(['kg', '1,5 litros', 'Litro', 'Docena', '750 cc', '1.000 cc',
       '125 g', '500 g'], dtype=object)

In [81]:
# Convertir la columna "Price" a numérica, forzando los errores a NaN
final_df["Price"] = pd.to_numeric(final_df["Price"], errors="coerce")

# Mostrar las filas con valores faltantes
missing_values = final_df[final_df["Price"].isna()]
missing_values

,Date,Region,Product,Unit,Price,product_id
6693,2020-03-01,Patagonia,Carne picada común,kg,NaN,6694
6963,2020-03-01,Patagonia,Leche fresca entera sachet,Litro,NaN,6964
7053,2020-03-01,Patagonia,Huevos de gallina,Docena,NaN,7054
7323,2020-03-01,Patagonia,Detergente líquido,750 cc,NaN,7324
7413,2020-03-01,Patagonia,Lavandina,1.000 cc,NaN,7414
7503,2020-03-01,Patagonia,Jabón de tocador,125 g,NaN,7504


### Podemos identificar que los errores estan en Marzo de 2020. en solo algunos productos, vamos a sacar la adyacente de los meses anteriores y subsiguientes para rellenar con valores cercanos de esos productos y regiones continuando la tendencia de los datos.

In [87]:
def fill_missing_with_adjacent(df):
    """
    Rellena los valores faltantes en la columna "Price" de un DataFrame
    utilizando los precios de los meses adyacentes para el mismo producto,
    región y unidad.

    Args:
        df (pd.DataFrame): DataFrame que contiene las columnas "Date",
                           "Region", "Product", "Unit" y "Price".

    Returns:
        pd.DataFrame: DataFrame con los valores faltantes en "Price"
                      rellenados con los precios promedios de los meses
                      adyacentes.
    """
    # Ordenar el DataFrame por "Date" para asegurar que los datos estén en orden cronológico
    df = df.sort_values(by="Date")
    
    # Iterar sobre cada fila con valores faltantes
    for index, row in df[df["Price"].isna()].iterrows():
        region = row["Region"]
        product = row["Product"]
        unit = row["Unit"]
        date = row["Date"]
        
        # Filtrar el DataFrame para obtener solo los datos del mismo producto y región
        product_data = df[(df["Region"] == region) & (df["Product"] == product) & (df["Unit"] == unit)]
        
        # Obtener los precios del mes anterior y el mes siguiente
        previous_price = product_data[product_data["Date"] < date]["Price"].dropna().tail(1)
        next_price = product_data[product_data["Date"] > date]["Price"].dropna().head(1)
        
        # Calcular el promedio de los valores disponibles
        if not previous_price.empty and not next_price.empty:
            df.at[index, "Price"] = (previous_price.iloc[0] + next_price.iloc[0]) / 2  # Promedio
        elif not previous_price.empty:
            df.at[index, "Price"] = previous_price.iloc[0]  # Usar solo el mes anterior
        elif not next_price.empty:
            df.at[index, "Price"] = next_price.iloc[0]  # Usar solo el mes siguiente
    
    return df

# Aplicar la función para rellenar valores faltantes
final_df = fill_missing_with_adjacent(final_df)
# Redondear la columna "Price" a 2 decimales
final_df["Price"] = final_df["Price"].round(2)

# Verificar si aún hay valores faltantes
print(final_df[final_df["Price"].isna()])

Empty DataFrame
Columns: [Date, Region, Product, Unit, Price, product_id]
Index: []


In [83]:
# Convertir la columna "Date" a tipo datetime, forzando los errores a NaN
final_df["Date"] = pd.to_datetime(final_df["Date"], errors="coerce")

# Mostrar las filas con valores faltantes
missing_values = final_df[final_df["Date"].isna()]
print(missing_values)
# Valores Correctos.

Empty DataFrame
Columns: [Date, Region, Product, Unit, Price, product_id]
Index: []


### Verificar Outliers

In [88]:
# Obtener los 5 valores más bajos en "Price"
lowest_prices = final_df.nsmallest(5, "Price")
print("5 valores más bajos en Price:")
print(lowest_prices)

# Obtener los 5 valores más altos en "Price"
highest_prices = final_df.nlargest(5, "Price")
print("\n5 valores más altos en Price:")
print(highest_prices)

5 valores más bajos en Price:
           Date Region                Product Unit  Price  product_id
5131 2017-07-01   Cuyo  Harina de trigo común   kg  10.28        5132
5130 2017-06-01   Cuyo  Harina de trigo común   kg  10.45        5131
5132 2017-08-01   Cuyo  Harina de trigo común   kg  10.46        5133
5133 2017-09-01   Cuyo  Harina de trigo común   kg  10.49        5134
5135 2017-11-01   Cuyo  Harina de trigo común   kg  10.51        5136

5 valores más altos en Price:
           Date     Region             Product Unit    Price  product_id
6749 2024-11-01  Patagonia  Carne picada común   kg  6429.10        6750
6748 2024-10-01  Patagonia  Carne picada común   kg  6106.38        6749
6747 2024-09-01  Patagonia  Carne picada común   kg  6018.02        6748
6741 2024-03-01  Patagonia  Carne picada común   kg  5797.99        6742
6746 2024-08-01  Patagonia  Carne picada común   kg  5732.88        6747


Tabla listo con validaciones y tipos de datos correctos

In [89]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7560 entries, 0 to 7559
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        7560 non-null   datetime64[ns]
 1   Region      7560 non-null   object        
 2   Product     7560 non-null   object        
 3   Unit        7560 non-null   object        
 4   Price       7560 non-null   float64       
 5   product_id  7560 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 413.4+ KB


### Create and Load at Alphacast

In [1410]:
from alphacast import Alphacast
alphacast = Alphacast('ak_EZGOL3ajzMuwiwBV3fTT')

In [1411]:
repo_name = "reto-tecnico" 
repo_description = "Prueba tecnica IPC completada"  
repo_slug = "reto-tecnico-mauricioarce" 
repo_privacy = "Public"  

# Crear el repositorio
repositorio = alphacast.repository.create(
    repo_name,
    repo_description=repo_description,
    slug=repo_slug,
    privacy=repo_privacy,
    returnIdIfExists=True 
)
print(f"Repositorio creado: {repositorio}")


Repositorio creado: {'id': 16342, 'name': 'reto-tecnico', 'description': 'Prueba tecnica IPC completada', 'privacy': 'Public', 'slug': 'reto-tecnico-16342'}


In [1419]:
# Verificacion
alphacast.repository.read_by_id(16342)

{'id': 16342,
 'name': 'reto-tecnico',
 'accountId': 8751,
 'description': 'Prueba tecnica IPC completada',
 'privacy': 'Public',
 'slug': 'reto-tecnico-16342',
 'permission': 'Owner'}

In [1420]:
# Crear dataset
dataset_name = 'prueba-tecnica-completada'
repo_id = 16342
description = 'Índice de precios al consumidor. Precios promedio de un conjunto de elementos de la canasta del IPC, según regiones (Junio de 2017-noviembre de 2024) y para el GBA (Abril de 2016-noviembre de 2024)'

alphacast.datasets.create(dataset_name, repo_id, description)


ValueError: Dataset already exists: 43861

In [1421]:
dataset_id = 43861
df = final_df[['Date', 'Region', 'Product', 'Unit', 'Price', 'product_id']]

# Inicializar columnas en Alphacast
alphacast.datasets.dataset(dataset_id).initialize_columns(
    dateColumnName="Date", 
    entitiesColumnNames=["product_id"], 
    dateFormat="%Y-%m-%d"
)

b'{"id": 43861, "columnDefinitions": [{"sourceName": "Date", "dataType": "Date", "dateFormat": "%Y-%m-%d", "isEntity": "True"}, {"sourceName": "product_id", "isEntity": "True"}], "updateAt": "2025-01-10T05:47:31.934318"}'

In [1422]:
alphacast.datasets.dataset(43861).upload_data_from_df(df, deleteMissingFromDB = False, onConflictUpdateDB = False, uploadIndex=False)

b'{"id": 5148773, "status": "Requested", "createdAt": "2025-01-10T05:48:16.252345", "datasetId": 43861}'

In [1423]:
alphacast.datasets.dataset(43861).processes()

b'[{"id": 5148773, "datasetId": 43861, "status": "Processed", "statusDescription": "7560 values added to database./n", "deleteMissingFromDB": 0, "onConflictUpdateDB": 0, "createdAt": "2025-01-10T05:48:16", "processedAt": "2025-01-10T05:48:19"}]'